In [1]:
import numpy as np
import csv
from math import pi,sin
import matplotlib.pyplot as plt

In [2]:
nx,ny,nz = 8,8,8

nstx, nsty, nstz = 1, 1, 1

nxg,nyg,nzg = (nx + 2*nstx), (ny + 2*nsty), (nz + 2*nstz)

nsrx,nsry,nsrz = 4,4,4

dx,dy,dz = (2*pi)/nx, (2*pi)/ny, (2*pi)/nz


fsp = 2 # u,u_old
dim = 5 # u, v, w, r, e
fs = 2 # x,u

In [3]:
points = [[[[[[0 for _ in range(fs)] for _ in range(fsp)] for _ in range(dim)] for i in range(nxg)] for _ in range(nyg)] for _ in range(nzg)]

In [4]:
points = np.array(points)

In [5]:
np.shape(points)

(10, 10, 10, 5, 2, 2)

In [6]:
def init_grid(points,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                ir,jr,kr = float(i),float(j),float(k)
                points[i][j][k][0][0][0] = (ir-nstx)*dx
                points[i][j][k][1][0][0] = (jr-nsty)*dy
                points[i][j][k][2][0][0] = (kr-nstz)*dz
                points[i][j][k][3][0][0] = 1
                points[i][j][k][4][0][0] = 1

In [7]:
def init_cond(points,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                points[i][j][k][0][1][1] = sin(points[i][j][k][0][0][0])
                points[i][j][k][1][1][1] = sin(points[i][j][k][1][0][0])
                points[i][j][k][2][1][1] = sin(points[i][j][k][2][0][0])
                points[i][j][k][3][1][1] = 1
                points[i][j][k][4][1][1] = 1

In [8]:
def time_step(dx,time,cfl,c,alpha,t_end,exitFlag):
    dt_d = (cfl*dx*dx)/alpha
    dt_c = (cfl*dx)/c
    
    dt = min(dt_d,dt_c)
    
    if time+dt>t_end:
        dt = t_end-time
        exitFlag = True
    time = time+dt
#     dt = dt
#     exitFlag = exitFlag
#     time = time
    return dt,exitFlag,time

In [9]:
def bc_update(points,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(1,nstx+1):
                points[nx1-i][j][k][0][1][1] = points[nx2-i+1][j][k][0][1][1]
                points[nx2+i][j][k][0][1][1] = points[nx1+i-1][j][k][0][1][1]
    for k in range(nz1,nz2+1):
        for j in range(1,nsty+1):
            for i in range(nx1,nx2+1):
                points[i][ny1-j][k][1][1][1] = points[i][ny2-j+1][k][1][1][1]
                points[i][ny2+j][k][1][1][1] = points[i][ny1+j-1][k][1][1][1]
    for k in range(1,nstz+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                points[i][j][nz1-k][2][1][1] = points[i][j][nz2-k+1][2][1][1]
                points[i][j][nz2+k][2][1][1] = points[i][j][nz1+k-1][2][1][1]

In [10]:
def buffer_update(points,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                points[i][j][k][0][0][1] = points[i][j][k][0][1][1]
                points[i][j][k][1][0][1] = points[i][j][k][1][1][1]
                points[i][j][k][2][0][1] = points[i][j][k][2][1][1]

In [11]:
def cp_sd(pts1,pts2,d):
    return (pts1-pts2)/(2.0*d)
def cp_td(pts1,pts2,pts3,d):
    return (pts1-2.0*pts2+pts3)/(d*d)
def cp_final(pts1,c,alpha,dt,d1x,d1y,d1z,d2x,d2y,d2z):
    return pts1-c*dt*(d1x+d1y+d1z)+alpha*dt*(d2x+d2y+d2z)

In [12]:
def compute_step(pts,dx,dy,dz,dt,c,alpha,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                du_dx = cp_sd(pts[i+1][j][k][0][0][1],pts[i-1][j][k][0][0][1],dx)
                du_dy = cp_sd(pts[i+1][j][k][0][0][1],pts[i-1][j][k][0][0][1],dy)
                du_dz = cp_sd(pts[i+1][j][k][0][0][1],pts[i-1][j][k][0][0][1],dz)
                dv_dx = cp_sd(pts[i][j+1][k][1][0][1],pts[i][j-1][k][1][0][1],dx)
                dv_dy = cp_sd(pts[i][j+1][k][1][0][1],pts[i][j-1][k][1][0][1],dy)
                dv_dz = cp_sd(pts[i][j+1][k][1][0][1],pts[i][j-1][k][1][0][1],dz)
                dw_dx = cp_sd(pts[i][j][k+1][2][0][1],pts[i][j][k-1][2][0][1],dx)
                dw_dy = cp_sd(pts[i][j][k+1][2][0][1],pts[i][j][k-1][2][0][1],dy)
                dw_dz = cp_sd(pts[i][j][k+1][2][0][1],pts[i][j][k-1][2][0][1],dz)
                du2_dx2 = cp_td(pts[i+1][j][k][0][0][1],pts[i][j][k][0][0][1],pts[i-1][j][k][0][0][1],dx)
                du2_dy2 = cp_td(pts[i+1][j][k][0][0][1],pts[i][j][k][0][0][1],pts[i-1][j][k][0][0][1],dy)
                du2_dz2 = cp_td(pts[i+1][j][k][0][0][1],pts[i][j][k][0][0][1],pts[i-1][j][k][0][0][1],dz)
                dv2_dx2 = cp_td(pts[i][j+1][k][1][0][1],pts[i][j][k][1][0][1],pts[i][j-1][k][1][0][1],dx)
                dv2_dy2 = cp_td(pts[i][j+1][k][1][0][1],pts[i][j][k][1][0][1],pts[i][j-1][k][1][0][1],dy)
                dv2_dz2 = cp_td(pts[i][j+1][k][1][0][1],pts[i][j][k][1][0][1],pts[i][j-1][k][1][0][1],dz)
                dw2_dx2 = cp_td(pts[i][j][k+1][2][0][1],pts[i][j][k][2][0][1],pts[i][j][k-1][2][0][1],dx)
                dw2_dy2 = cp_td(pts[i][j][k+1][2][0][1],pts[i][j][k][2][0][1],pts[i][j][k-1][2][0][1],dy)
                dw2_dz2 = cp_td(pts[i][j][k+1][2][0][1],pts[i][j][k][2][0][1],pts[i][j][k-1][2][0][1],dz)
                
                pts[i][j][k][0][1][1] = cp_final(pts[i][j][k][0][0][1],c,alpha,dt,du_dx,du_dy,du_dz,du2_dx2,du2_dy2,du2_dz2)
                pts[i][j][k][1][1][1] = cp_final(pts[i][j][k][1][0][1],c,alpha,dt,dv_dx,dv_dy,dv_dz,dv2_dx2,dv2_dy2,dv2_dz2)
                pts[i][j][k][2][1][1] = cp_final(pts[i][j][k][2][0][1],c,alpha,dt,dw_dx,dw_dy,dw_dz,dw2_dx2,dw2_dy2,dw2_dz2)

In [19]:
def write_data(pts,nx,ny,nz,nstx,nsty,nstz):
    nx1,ny1,nz1 = nstx,nsty,nstz
    
    nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
    up,vp,wp=[],[],[]
    for k in range(nz1,nz2+1):
        for j in range(ny1,ny2+1):
            for i in range(nx1,nx2+1):
                print(pts[i][j][k][0][1][1])
#                 up.append(pts[i][j][k][0][1][1])
#                 vp.append(pts[i][j][k][1][1][1])
#                 wp.append(pts[i][j][k][2][1][1])
    return up,vp,wp

In [14]:
init_grid(points,nx,ny,nz,nstx,nsty,nstz)

In [15]:
points

array([[[[[[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]]],


         [[[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]]],


         [[[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]]],


         ...,


         [[[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]]],


         [[[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0],
           [0, 0]],

          [[0, 0

In [16]:
init_cond(points,nx,ny,nz,nstx,nsty,nstz)

In [20]:
time=0.0
exitFlag=False
t_end=0.5
alpha=1.0
cfl=0.1
c=1.0
nt=2

for i in range(1,nt):
    bc_update(points,nx,ny,nz,nstx,nsty,nstz)
    buffer_update(points,nx,ny,nz,nstx,nsty,nstz)
    dt,exitFlag,time = time_step(dx,time,cfl,c,alpha,t_end,exitFlag)
    compute_step(points,dx,dy,dz,dt,c,alpha,nx,ny,nz,nstx,nsty,nstz)
    up,vp,wp = write_data(points,nx,ny,nz,nstx,nsty,nstz)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [18]:
up,vp,wp

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [94]:
class driver:
    def __init__(self,nxg,nyg,nzg,fs,fsp,dim):
        self.points = [[[[[[0 for _ in range(fs)] for _ in range(fsp)] for _ in range(dim)] for i in range(nxg)] for _ in range(nyg)] for _ in range(nzg)]
#         self.points = np.array(self.points)
        
    def init_grid(self,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
        
        dx,dy,dz = (2*pi)/nx, (2*pi)/ny, (2*pi)/nz
        
        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    ir,jr,kr = float(i),float(j),float(k)
                    self.points[i][j][k][0][0][0] = (ir-nstx)*dx
                    self.points[i][j][k][1][0][0] = (jr-nsty)*dy
                    self.points[i][j][k][2][0][0] = (kr-nstz)*dz
                    self.points[i][j][k][3][0][0] = 1
                    self.points[i][j][k][4][0][0] = 1
                    
    def init_cond(self,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1

        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    self.points[i][j][k][0][1][1] = sin(self.points[i][j][k][0][0][0])
                    self.points[i][j][k][1][1][1] = sin(self.points[i][j][k][1][0][0])
                    self.points[i][j][k][2][1][1] = sin(self.points[i][j][k][2][0][0])
                    self.points[i][j][k][3][1][1] = 1
                    self.points[i][j][k][4][1][1] = 1

                    
    def time_step(self,dx,time,cfl,c,alpha,t_end,exitFlag):
        dt_d = (cfl*dx*dx)/alpha
        dt_c = (cfl*dx)/c

        dt = min(dt_d,dt_c)

        if time+dt>t_end:
            dt = t_end-time
            exitFlag = True
        time = time+dt
    #     dt = dt
    #     exitFlag = exitFlag
    #     time = time
        return dt,exitFlag,time

    def bc_update(self,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1

        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(1,nstx+1):
                    self.points[nx1-i][j][k][0][1][1] = self.points[nx2-i+1][j][k][0][1][1]
                    self.points[nx2+i][j][k][0][1][1] = self.points[nx1+i-1][j][k][0][1][1]
        for k in range(nz1,nz2+1):
            for j in range(1,nsty+1):
                for i in range(nx1,nx2+1):
                    self.points[i][ny1-j][k][1][1][1] = self.points[i][ny2-j+1][k][1][1][1]
                    self.points[i][ny2+j][k][1][1][1] = self.points[i][ny1+j-1][k][1][1][1]
        for k in range(1,nstz+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    self.points[i][j][nz1-k][2][1][1] = self.points[i][j][nz2-k+1][2][1][1]
                    self.points[i][j][nz2+k][2][1][1] = self.points[i][j][nz1+k-1][2][1][1]
                    
    def buffer_update(self,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1

        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    self.points[i][j][k][0][0][1] = self.points[i][j][k][0][1][1]
                    self.points[i][j][k][1][0][1] = self.points[i][j][k][1][1][1]
                    self.points[i][j][k][2][0][1] = self.points[i][j][k][2][1][1]
                    
    def cp_sd(self,pts1,pts2,d):
        return (pts1-pts2)/(2.0*d)
    def cp_td(self,pts1,pts2,pts3,d):
        return (pts1-2.0*pts2+pts3)/(d*d)
    def cp_final(self,pts1,c,alpha,dt,d1x,d1y,d1z,d2x,d2y,d2z):
        return pts1-c*dt*(d1x+d1y+d1z)+alpha*dt*(d2x+d2y+d2z)
    
    def compute_step(self,dx,dy,dz,dt,c,alpha,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1

        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    du_dx = self.cp_sd(self.points[i+1][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dx)
                    du_dy = self.cp_sd(self.points[i+1][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dy)
                    du_dz = self.cp_sd(self.points[i+1][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dz)
                    dv_dx = self.cp_sd(self.points[i][j+1][k][1][0][1],self.points[i][j-1][k][1][0][1],dx)
                    dv_dy = self.cp_sd(self.points[i][j+1][k][1][0][1],self.points[i][j-1][k][1][0][1],dy)
                    dv_dz = self.cp_sd(self.points[i][j+1][k][1][0][1],self.points[i][j-1][k][1][0][1],dz)
                    dw_dx = self.cp_sd(self.points[i][j][k+1][2][0][1],self.points[i][j][k-1][2][0][1],dx)
                    dw_dy = self.cp_sd(self.points[i][j][k+1][2][0][1],self.points[i][j][k-1][2][0][1],dy)
                    dw_dz = self.cp_sd(self.points[i][j][k+1][2][0][1],self.points[i][j][k-1][2][0][1],dz)
                    du2_dx2 = self.cp_td(self.points[i+1][j][k][0][0][1],self.points[i][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dx)
                    du2_dy2 = self.cp_td(self.points[i+1][j][k][0][0][1],self.points[i][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dy)
                    du2_dz2 = self.cp_td(self.points[i+1][j][k][0][0][1],self.points[i][j][k][0][0][1],self.points[i-1][j][k][0][0][1],dz)
                    dv2_dx2 = self.cp_td(self.points[i][j+1][k][1][0][1],self.points[i][j][k][1][0][1],self.points[i][j-1][k][1][0][1],dx)
                    dv2_dy2 = self.cp_td(self.points[i][j+1][k][1][0][1],self.points[i][j][k][1][0][1],self.points[i][j-1][k][1][0][1],dy)
                    dv2_dz2 = self.cp_td(self.points[i][j+1][k][1][0][1],self.points[i][j][k][1][0][1],self.points[i][j-1][k][1][0][1],dz)
                    dw2_dx2 = self.cp_td(self.points[i][j][k+1][2][0][1],self.points[i][j][k][2][0][1],self.points[i][j][k-1][2][0][1],dx)
                    dw2_dy2 = self.cp_td(self.points[i][j][k+1][2][0][1],self.points[i][j][k][2][0][1],self.points[i][j][k-1][2][0][1],dy)
                    dw2_dz2 = self.cp_td(self.points[i][j][k+1][2][0][1],self.points[i][j][k][2][0][1],self.points[i][j][k-1][2][0][1],dz)

                    self.points[i][j][k][0][1][1] = self.cp_final(self.points[i][j][k][0][0][1],c,alpha,dt,du_dx,du_dy,du_dz,du2_dx2,du2_dy2,du2_dz2)
                    self.points[i][j][k][1][1][1] = self.cp_final(self.points[i][j][k][1][0][1],c,alpha,dt,dv_dx,dv_dy,dv_dz,dv2_dx2,dv2_dy2,dv2_dz2)
                    self.points[i][j][k][2][1][1] = self.cp_final(self.points[i][j][k][2][0][1],c,alpha,dt,dw_dx,dw_dy,dw_dz,dw2_dx2,dw2_dy2,dw2_dz2)
    def error(self):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
        sumprod=0
        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
        
                    uprod = self.points[i][j][k][0][1][1]*self.points[i][j][k][0][1][1]
                    vprod = self.points[i][j][k][1][1][1]*self.points[i][j][k][1][1][1]
                    wprod = self.points[i][j][k][2][1][1]*self.points[i][j][k][2][1][1]
                    sumprod += (uprod+vprod+wprod)/2
        return sumprod
                    
    def write_data(self,nx,ny,nz,nstx,nsty,nstz):
        nx1,ny1,nz1 = nstx,nsty,nstz

        nx2,ny2,nz2 = nx+nstx-1,ny+nsty-1,nz+nstz-1
        up,vp,wp=[],[],[]
        for k in range(nz1,nz2+1):
            for j in range(ny1,ny2+1):
                for i in range(nx1,nx2+1):
                    up.append(self.points[i][j][k][0][1][1])
                    vp.append(self.points[i][j][k][1][1][1])
                    wp.append(self.points[i][j][k][2][1][1])
        return up,vp,wp

In [95]:
nx,ny,nz = 8,8,8

nstx, nsty, nstz = 1, 1, 1

nxg,nyg,nzg = (nx + 2*nstx), (ny + 2*nsty), (nz + 2*nstz)

nsrx,nsry,nsrz = 4,4,4

dx,dy,dz = (2*pi)/nx, (2*pi)/ny, (2*pi)/nz


fsp = 2 # u,u_old
dim = 5 # u, v, w, r, e
fs = 2 # x,u

time=0.0
exitFlag=False
t_end=0.5
alpha=1.0
cfl=0.1
c=1.0
nt=2

solver = driver(nxg,nyg,nzg,fs,fsp,dim)

solver.init_grid(nx,ny,nz,nstx,nsty,nstz)
solver.init_cond(nx,ny,nz,nstx,nsty,nstz)

for i in range(1,nt):
    sum_=0
    solver.bc_update(nx,ny,nz,nstx,nsty,nstz)
    solver.buffer_update(nx,ny,nz,nstx,nsty,nstz)
    dt,exitFlag,time = solver.time_step(dx,time,cfl,c,alpha,t_end,exitFlag)
    solver.compute_step(dx,dy,dz,dt,c,alpha,nx,ny,nz,nstx,nsty,nstz)
    sum1 = solver.error()
    sum_ += sum1
    print("T: {}, Sum: {}\n".format(i,sum_))
    
    if exitFlag == True:
        break
up,vp,wp = solver.write_data(nx,ny,nz,nstx,nsty,nstz)

T: 1, Sum: 270.48157970438365



In [88]:
ur,vr,wr=[],[],[]
with open('Solution_0_0_0.txt','r') as csvfile:
    files = csv.reader(csvfile,delimiter="\t")
    files = list(files)
    for file in files:
        ur.append(float(file[0]))
        vr.append(float(file[1]))
        wr.append(float(file[2]))

In [89]:
ur

[-0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,
 -0.166018,
 -0.245259,
 -0.180831,
 -0.010475,
 0.166018,
 0.245259,
 0.180831,
 0.010475,


In [96]:
up

[0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.8242640687119286,
 -0.7006524369842363,
 0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.8242640687119286,
 -0.7006524369842363,
 0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.8242640687119286,
 -0.7006524369842363,
 0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.8242640687119286,
 -0.7006524369842363,
 0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.8242640687119286,
 -0.7006524369842363,
 0.12882797926549489,
 0.4650329879650017,
 0.8242640687119285,
 0.7006524369842363,
 0.16660811018093882,
 -0.4650329879650017,
 -0.82